In [1]:
import os
from glob import glob

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
question = "대학생에게 필요한 금융 지식이 뭘까?"
model_name = "BAAI/bge-m3"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

test_embedding = embeddings.embed_query(question)
len(test_embedding)

model.safetensors:  11%|#         | 241M/2.27G [00:00<?, ?B/s]

1024

In [3]:
lst_pdf = glob("../../Data/PDF/*.pdf")

# 1️⃣ PDF 파일 로드
pdf_path = lst_pdf[0]
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# 2️⃣ 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=400)
splits = text_splitter.split_documents(documents)

# 3️⃣ 결과 출력
print(f"총 {len(splits)} 개의 청크로 분할됨.")
print(splits[0].page_content)

vectordb = FAISS.from_documents(documents=splits, embedding=embeddings)

총 948 개의 청크로 분할됨.
대학생을 위한
실용금융
제4판


In [4]:
import torch
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8비트 로딩 (or load_in_4bit=True for more compression)
    llm_int8_threshold=6.0
)

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    truncation=True,
    max_length="longest_first"
)
tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    quantization_config=bnb_config
)

hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.7,
    top_p=0.9,
    device_map="auto"
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [19]:
retriever.generate_queries(question)

TypeError: MultiQueryRetriever.generate_queries() missing 1 required positional argument: 'run_manager'

In [5]:
from uuid import uuid4

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.retrievers import MultiQueryRetriever


handler = BaseCallbackHandler()

run_id = uuid4()
run_manager = CallbackManagerForRetrieverRun(
    run_id=run_id,
    handlers=[handler],
    inheritable_handlers=[]
)

retriever = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(),
    llm=llm,
    include_original=True
)

multiQueries = retriever.generate_queries(question, run_manager=run_manager)
multiQueries

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['You are an AI language model assistant. Your task is ',
 '    to generate 3 different versions of the given user ',
 '    question to retrieve relevant documents from a vector  database. ',
 '    By generating multiple perspectives on the user question, ',
 '    your goal is to help the user overcome some of the limitations ',
 '    of distance-based similarity search. Provide these alternative ',
 '    questions separated by newlines. Original question: 대학생에게 필요한 금융 지식이 뭘까? (What financial knowledge do college students need?)',
 ' ',
 ' ',
 ' ',
 '1. 대학생에게 필요한 금융 지식은 무엇인가? (What financial knowledge do college students need?)',
 '2. 대학생들이 금융 지식에 대해 어떤 질문을 할 수 있을까요? (What questions do college students ask about financial knowledge?)',
 '3. 대학생들이 금융 지식에 대해 어떤 문제를 해결할 수 있을까요? (What problems can college students solve with financial knowledge?)',
 ' ',
 ' ',
 '4. 대학생들에게 필요한 금융 지식은 어떤 것이 있을까요? (What financial knowledge do college students need?)',
 '5. 대학생들이 금융 지식에 대해 어떤 도구나 리소스를 활용할 수 있을

In [6]:
# MultiQueryRetriever 설정
retriever = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(),
    llm=llm,
    include_original=True
)

# 테스트 실행
retrieved_docs = retriever.invoke(question)

for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1}: {doc.page_content}\n")
    if i==3:
        break

Document 1: 제12장  신기술과 금융의 변화 425
핀테크 관련 주요 기술12.2
금융을 변화시키는 다양한 첨단기술이 있는데 핀테크에서는 시스템을 설계하고 제어
할 수 있는 인프라 소프트웨어 기술이 중심이 된다. 대표적인 기술로서 인공지능, 블록
체인, 클라우드, 빅데이터를 꼽을 수 있는데, 각 기술 명칭의 제일 앞 글자를 따서 이를 
ABCD기술이라고 지칭하기도 한다. 
인공지능1
인공지능(AI, Artificial Intelligence)이란 인간의 지능으로 할 수 있는 사고, 학습, 자기 개
발 등을 컴퓨터가 할 수 있도록 하는 컴퓨터 공학 및 정보기술의 한 분야이다. 즉, 딥러닝
(Deep Learning, 심층학습)을 기반으로 대규모의 데이터를 축적하여 인간과 유사한 학습, 지
각, 판단 능력 등을 구현한 것이며, 데이터 분석기술과 컴퓨터 성능이 발달하며 점차 완성
도가 높아지고 있다.
인공지능은 챗봇4을 기초로 금융회사의 고객 상담, 금융상품 추천 등 고객 응대 업무에 
활용되고 있다. 또한 인공지능이 빅데이터 및 투자자 성향을 분석하여 자산관리 포트폴리
오를 설계하여 제공하거나, 여신심사 관련 금융데이터를 분석하여 개인 신용평가의 정밀
도를 높이는 사례도 있다. 이 외에도 인공지능 기술을 활용하여 금융사기 또는 이상거래
를 탐지하는 체계를 구축하는 등 금융권에서의 활용이 확산되고 있다. 
4  챗봇이란 고객이 입력한 문자나 말한 음성내용을 인공지능을 활용하여 인식하고 고객의 응대에 자동적으
로 답하는 기능을 말한다.

Document 2: 438  대학생을 위한 실용금융
웰스테크4
웰스테크(WealthTech)는 부(wealth)와 기술(tech)의 합성어로, 자산관리를 위한 핀테크 서
비스를 지칭한다. 가장 대표적인 분야로 금융투자 관련 로보어드바이저(Robo–Advisor)를 
들 수 있다. 
로보어드바이저는 로봇(Robot)과 투자자문가(Advisor)의 합성어이며, 인공지능을 활용하
여 투자자의 리스크 관련 성향, 목표수익률, 자금

In [7]:
len(retrieved_docs)

33